In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()


client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [2]:
import requests

SYSTEM_PROMPT = f"""당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.
실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)
추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.
당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.
"""

MESSAGES = [
    {
        'role': 'system',
        'content': SYSTEM_PROMPT
    }
]

def recommend(query_text):
    url = "http://localhost:8000/recommend"
    response = requests.post(url, json={"query_text": query_text})
    return response.json()


tools = [
    {
        "type": "function",
        "function": {
            "name": "recommend",
            "description": "사용자 발화 기반으로 메뉴 추천 API를 호출합니다. 오로지 이 함수 결과로만 메뉴 추천되어야 합니다. 현재 해장 또는 다이어트 2개 카테고리에 대한 메뉴 추천만 가능하며, 사용자 발화가 없는 경우 빈 리스트가 반환 될 수 있습니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query_text": {
                        "type": "string",
                        "description": "사용자 발화 텍스트 원문",
                    },
                },
                "required": ["query_text"],
                "additionalProperties": False,
            },
        }
    }
]

In [3]:
MESSAGES.append(
    {"role": "user", "content": "안녕"}
)

completion = client.chat.completions.create(
    messages=MESSAGES,
    model="gpt-4o-2024-08-06",
    tools=tools
)

completion

ChatCompletion(id='chatcmpl-COfo0gj075gCcIDjWNsxnYT4yJxsi', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="안녕하세요! 배달의민족 메뉴 추천 서비스 '메뉴뚝딱AI' 입니다. 어떤 음식을 찾고 계신가요? 해장에 좋은 음식이나 다이어트에 도움되는 메뉴를 추천해드릴 수 있습니다. 원하시는 메뉴 카테고리를 말씀해주시면 추천해드릴게요!", refusal=None, role='assistant', function_call=None, tool_calls=None, annotations=[]))], created=1759996608, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_f33640a400', usage=CompletionUsage(completion_tokens=73, prompt_tokens=244, total_tokens=317, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}))

In [4]:
if completion.choices[0].finish_reason == "tool_calls":
    print('function calling lets go')
else:
    MESSAGES.append(
        {"role": "assistant", "content": completion.choices[0].message.content}
    )

In [5]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant',
  'content': "안녕하세요! 배달의민족 메뉴 추천 서비스 '메뉴뚝딱AI' 입니다. 어떤 음식을 찾고 계신가요? 해장에 좋은 음식이나 다이어트에 도움되는 메뉴를 추천해드릴 수 있습니다. 원하시는 메뉴 카테고리를 말씀해주시면 추천해드릴게요!"}]

In [6]:
MESSAGES.append(
    {"role": "user", "content": "다이어트 메뉴 추천해줄레?"}
)

completion = client.chat.completions.create(
    messages=MESSAGES,
    model="gpt-4o-2024-08-06",
    tools=tools
)

completion

ChatCompletion(id='chatcmpl-COfoExXnUapnUf3mPkcV4KWEAEJKM', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_DjoHDCdO2Z2g5Moh5DZxtF6F', function=Function(arguments='{"query_text":"다이어트 메뉴 추천"}', name='recommend'), type='function')], annotations=[]))], created=1759996622, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_f33640a400', usage=CompletionUsage(completion_tokens=19, prompt_tokens=333, total_tokens=352, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}))

In [7]:
import json

if completion.choices[0].finish_reason == "tool_calls":
    tool_calls = completion.choices[0].message.tool_calls
    tool_name = tool_calls[0].function.name
    tool_args = tool_calls[0].function.arguments
    tool_id = tool_calls[0].id
    print('function calling lets go')
else:
    MESSAGES.append(
        {"role": "assistant", "content": completion.choices[0].message.content}
    )

function calling lets go


In [8]:
completion.choices[0].message.tool_calls[0]

ChatCompletionMessageToolCall(id='call_DjoHDCdO2Z2g5Moh5DZxtF6F', function=Function(arguments='{"query_text":"다이어트 메뉴 추천"}', name='recommend'), type='function')

In [9]:
tool_result = recommend(**json.loads(tool_args))
tool_result

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /recommend (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1060b63c0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [18]:
tool_calls[0].id

'call_r5DXAutpO9fYm4Z2D7UtojTd'

In [19]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant', 'content': '안녕하세요! 메뉴 추천이 필요하신가요? 어떤 음식을 추천해드릴까요?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄레?'},
 {'role': 'user', 'content': '다이어트 메뉴 추천해줄레?'}]

In [20]:
MESSAGES.append({
    "role": "assistant",  # user does not work
    "content": None,
    "tool_calls": [
        {
            "id": tool_id,
            "type": "function",
            "function": {
                "name": tool_name,
                "arguments": tool_args
            }
        }
    ]
})

In [21]:
tool_args

'{"query_text": "해장 메뉴 추천해줘"}'

In [22]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant', 'content': '안녕하세요! 메뉴 추천이 필요하신가요? 어떤 음식을 추천해드릴까요?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄레?'},
 {'role': 'user', 'content': '다이어트 메뉴 추천해줄레?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_r5DXAutpO9fYm4Z2D7UtojTd',
    'type': 'function',
    'function': {'name': 'recommend',
     'arguments': '{"query_text": "해장 메뉴 추천해줘"}'}}]}]

In [24]:
tool_result

{'detail': 'Not Found'}

In [26]:
MESSAGES.append({
    "role": "tool",
    "content": json.dumps(tool_result, ensure_ascii=False),
    "tool_call_id": tool_id
})

In [27]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant', 'content': '안녕하세요! 메뉴 추천이 필요하신가요? 어떤 음식을 추천해드릴까요?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄레?'},
 {'role': 'user', 'content': '다이어트 메뉴 추천해줄레?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_r5DXAutpO9fYm4Z2D7UtojTd',
    'type': 'function',
    'function': {'name': 'recommend',
     'arguments': '{"query_text": "해장 메뉴 추천해줘"}'}}]},
 {'role': 'tool',
  'content': '{"detail": "Not Found"}',
  'tool_call_id': 'call_r5DXAutpO9fYm4Z2D7UtojTd'}]

In [28]:
completion = client.chat.completions.create(
    messages=MESSAGES,
    model="gpt-4o-2024-08-06",
    tools=tools
)
completion

ChatCompletion(id='chatcmpl-COfjRcmAMw9nPglN6xvjvSCehaiwJ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_6u3ErcUQ1KxKl7S7zVIvbRxh', function=Function(arguments='{"query_text": "해장 메뉴 추천해줘"}', name='recommend'), type='function'), ChatCompletionMessageToolCall(id='call_Wfcxx2N6fBziN1i6RsOREEOz', function=Function(arguments='{"query_text": "다이어트 메뉴 추천해줘"}', name='recommend'), type='function')], annotations=[]))], created=1759996325, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_f33640a400', usage=CompletionUsage(completion_tokens=57, prompt_tokens=328, total_tokens=385, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}))

In [29]:
MESSAGES.append(
    {"role": "assistant", "content": completion.choices[0].message.content}
)

In [30]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant', 'content': '안녕하세요! 메뉴 추천이 필요하신가요? 어떤 음식을 추천해드릴까요?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄레?'},
 {'role': 'user', 'content': '다이어트 메뉴 추천해줄레?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_r5DXAutpO9fYm4Z2D7UtojTd',
    'type': 'function',
    'function': {'name': 'recommend',
     'arguments': '{"query_text": "해장 메뉴 추천해줘"}'}}]},
 {'role': 'tool',
  'content': '{"detail": "Not Found"}',
  'tool_call_id': 'call_r5DXAutpO9fYm4Z2D7UtojTd'},
 {'role': 'assistant', 'content': None}]